In [ ]:
import pandas as pd
from scipy.stats import pearsonr
data = pd.read_csv("table.how_similar.nok.txt", sep='\t')

In [ ]:
data['bin_sz'] = pd.cut(data['szsim'], [0, 0.80, 0.85, 0.90, 0.95, 1.01], labels=["<0.8", "0.85", "0.90", "0.95", "1"])
data['bin_sim'] = pd.cut(data['unroll'], [0, 0.80, 0.85, 0.90, 0.95, 1.01], labels=["<0.8", "0.85", "0.90", "0.95", "1"])

In [ ]:
data['bin_len'] = pd.cut(data['sz1'], [0, 50, 100, 500, 1000, 900000], labels=["0", "50", "100", "1000", "big"])

In [ ]:
len(data)

In [ ]:
print(pearsonr(data['seqsim'], data['pj']))

In [ ]:
import seaborn as sb
p = sb.scatterplot(data=data, x='seqsim', y='pj', hue='bin_len')
p.set(title="Kmer estimate of sequence similarity", xlabel="Sequence Similarity", ylabel="Kmer Count Similarity")

In [ ]:
print(pearsonr(data['unroll'], data['pj']))

In [ ]:
import seaborn as sb
p = sb.scatterplot(data=data, x='unroll', y='pj', hue='bin_len')
p.set(title="Kmer estimate of unroll sequence similarity", xlabel="Unroll Sequence Similarity", ylabel="Kmer Count Similarity")